# Libraries

In [1]:
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

2023-05-04 14:45:24.142127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 14:45:27.763827: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-04 14:45:33.946339: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-04 14:45:33.946574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Building the Neural Network

In [2]:
def create_cnn(width=154, height=154, depth=3, filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(11)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model  

# Preprocessing and converting the data to train and test

In [3]:
base_path = 'dataset/'
data = {}
for classes in os.listdir(base_path):
    img_path = base_path+classes+'/'
    cls = int(classes)
    if  data.get(cls) is None:
        data[cls] = []

    for images in os.listdir(img_path):
        fmt = images.split('.')

        if fmt[-1] == "jpg":
            id = fmt[0]
            image_path = img_path + images
            img = Image.open(image_path)
            width, height = img.size
            top_crop = 66
            bottom_crop = 20
            left_crop = 43
            right_crop = 43 
            box = (left_crop, top_crop, width-right_crop, height - bottom_crop)
            cropped_img = img.crop(box)
            data[cls].append(np.array(cropped_img))
            img.close()

n_train = 4
Y_train = np.empty((0), int)
Y_test = np.empty((0), int)
X_train = np.empty((0,154,154,3), float)
X_test = np.empty((0,154,154,3), float)

for label in data:
    if label != 90:
        n = len(data[label])
        Y_train = np.append(Y_train, [label]*n_train, axis=0)
        X_train = np.append(X_train, data[label][0:n_train], axis=0)
        n -= n_train
        Y_test = np.append(Y_test, [label]*n, axis=0)
        X_test = np.append(X_test, data[label][n_train:], axis=0)
    else:
        dtr = np.array(data[label][0])
        dtr = dtr[np.newaxis,:]
        dte = np.array(data[label][1])
        dte = dte[np.newaxis,:]
        Y_train = np.append(Y_train, [label], axis=0)
        X_train = np.append(X_train, dtr, axis=0)
        Y_test = np.append(Y_test, [label], axis=0)
        X_test = np.append(X_test, dte, axis=0)

X_train = np.append(X_train, X_test, axis=0)
Y_train = np.append(Y_train, Y_test, axis=0)
X_test = X_train
Y_test = Y_train

Y = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y[Y_train[i]//10,i] = 1

# Training the model

In [4]:
model = create_cnn(154, 154, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=X_train, y=Y.T, epochs=500, batch_size=50)

2023-05-04 14:45:44.246003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-04 14:45:46.885492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-04 14:45:46.885702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-04 14:45:46.893916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Epoch 1/500


2023-05-04 14:46:05.816644: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8900


4/4 [==============================] - 36s 114ms/step - loss: 2.6698 - accuracy: 0.2611
Epoch 2/500
4/4 [==============================] - 0s 37ms/step - loss: 1.1104 - accuracy: 0.6178
Epoch 3/500
4/4 [==============================] - 0s 43ms/step - loss: 0.6686 - accuracy: 0.7834
Epoch 4/500
4/4 [==============================] - 0s 44ms/step - loss: 0.5512 - accuracy: 0.8217
Epoch 5/500
4/4 [==============================] - 0s 37ms/step - loss: 0.5115 - accuracy: 0.8471
Epoch 6/500
4/4 [==============================] - 0s 39ms/step - loss: 0.5117 - accuracy: 0.8408
Epoch 7/500
4/4 [==============================] - 0s 38ms/step - loss: 0.6202 - accuracy: 0.7962
Epoch 8/500
4/4 [==============================] - 0s 37ms/step - loss: 0.5364 - accuracy: 0.8217
Epoch 9/500
4/4 [==============================] - 0s 35ms/step - loss: 0.5103 - accuracy: 0.8153
Epoch 10/500
4/4 [==============================] - 0s 44ms/step - loss: 0.4319 - accuracy: 0.8535
Epoch 11/500
4/4 [===========

# Testing the trained model

In [5]:
preds = model.predict(X_test)
y_pred = np.zeros(len(preds))

for i in range(0,len(preds)):
    y_pred[i] = 10 * np.argmax(preds[i,:])

print("Confusion Matrix :")
print(confusion_matrix(Y_test, y_pred))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_test, y_pred)))

5/5 [==============================] - 1s 43ms/step
Confusion Matrix :
[[11  0  0  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0  0  0  0  0]
 [ 0  0 13  1  0  0  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0]
 [ 0  0  0  0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  0 22]]

Accuracy: 0.99

